# 2. 페르소나 챗봇 만들기

## 0) 환경변수 불러오기

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# 챗봇 1

## 1) 구성 요소 생성

### Model

In [3]:
from langchain_openai import ChatOpenAI

# Model 생성
model = ChatOpenAI(
    model="gpt-4o-mini",  # 비용 gpt-4.1-nano < gpt-4o-mini < gpt-4
    temperature=0.5       # 창의성 정도(0~1 사이의 값. 1에 가까울수록 창의적인 답변)
)
response = model.invoke("좋은 아침!")
print(response)

content='좋은 아침입니다! 오늘 하루도 잘 보내시길 바랍니다. 무엇을 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 12, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BbnmDEMA6RpTZGVDAIGbKnLOitxur', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--2bc81d7d-c335-4660-9ab7-79b48daa14bb-0' usage_metadata={'input_tokens': 12, 'output_tokens': 23, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### Prompt

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = "당신은 동기부여 된 AI 어시스턴트입니다. 사용자에게 정확하고 도움이 되는 답변을 제공하세요. 사용자의 질문을 영어로 번역해서 제공한 후에 영어로 답변합니다."

# 히스토리를 반영한 Prompt 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])

print(prompt)

input_variables=['history', 'input'] input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.functio

### OutputParser

In [5]:
from langchain_core.output_parsers import StrOutputParser

# OutputParser 생성
output_parser = StrOutputParser()

## 2) Runnable 만들기

In [6]:
# Runnable 생성
runnable = prompt | model | output_parser 

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory

# 히스토리가 저장될 변수 생성
store = {}  

# 세션 ID별 히스토리를 가져오는 함수
def get_session_history(session_id):
    # store의 key에 session_id가 없는 경우 session_id를 key에 추가
    if session_id not in store: 
        store[session_id] = ChatMessageHistory()
        
    # session_id 키의 값을 반환
    return store[session_id]

In [8]:
from langchain_core.runnables.history import RunnableWithMessageHistory

# Chain 생성
chain = (
    RunnableWithMessageHistory(          # RunnableWithMessageHistory 객체 생성
        runnable,                        # 실행할 Runnable 객체
        get_session_history,             # 세션 ID별 히스토리를 가져오는 함수
        input_messages_key="input",      # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

## 챗봇 1 테스트

In [9]:
# 무한 루프
while True:
    ## 사용자 입력 받기
    question = input("[USER] ")
    print(f"[USER]: {question}")
    ## 종료 조건
    if question == "q":
        print("대화를 종료합니다.")
        break
    ## 응답 받기
    response = chain.invoke(
        {"input": question},
        {"configurable": {"session_id": "user1"}}
    )
    print(f"[AI]: {response}")

[USER]: 안녕, 즐거운 밤이야!
[AI]: Hello, it's a pleasant night! 

I'm glad to hear that! How can I assist you tonight?
[USER]: 오늘 날씨가 좋더라!
[AI]: The weather was nice today!

That's great to hear! Nice weather can really lift your spirits. Did you get a chance to enjoy it outside?
[USER]: q
대화를 종료합니다.
